# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from IPython.core.display import HTML, display
import lxml
from lxml import html
import requests
import json
import geopy.distance
from functools import partial
from shapely.geometry import Point
import shapely
from tqdm import tqdm
from pprint import pprint

# Scrape Data

## Set up Onemap API

In [ ]:
email = "" # fill email for onemap api
pw = "" # fill password for onemap api

#https://github.com/chengguan/pyonemap/blob/main/pyonemap/onemap.py
__author__ = "Teo Cheng Guan"
__copyright__ = "Copyright (C) 2023 Teo Cheng Guan"
__license__ = "MIT"

import requests

class Core:
    def __init__(self, api_key=None):
        self.__base_url = "https://www.onemap.gov.sg"
        self.__api_key = api_key

    def setToken(self, access_token):
        self.__api_key = access_token

    def getToken(email, password):
        url = "https://www.onemap.gov.sg/api/auth/post/getToken"
        payload = {"email": email, "password": password}

        try:
            response = requests.request("POST", url, json=payload)
            response.raise_for_status()  # Check for HTTP errors
        except requests.exceptions.RequestException as e:
            # Handle any exceptions that occurred during the request
            print(f"An error occurred: {e}")
            return None

        return response.json()

    def make_request(self, endpoint, params):
        url = f"{self.__base_url}{endpoint}"
        headers = {}

        if self.__api_key:
            headers["Authorization"] = f"Bearer {self.__api_key}"

        response = requests.get(url, params=params, headers=headers)

        if response.status_code == 200:
            return response.json()
        else:
            response.raise_for_status()


class OneMap:
    def __init__(self, api_key=None):
        self.__core = Core(api_key)
        #self.routing = Routing(api_key)

    def getToken(email, password):
        return Core.getToken(email, password)

    '''
        searchVal string REQUIRED
        Keywords entered by users to filter the results

        returnGeom string REQUIRED
        Values: Y, N . Enter Y if user wants the geometry value returned.

        getAddrDetails string REQUIRED
        Values: Y, N . Enter Y if user wants address details returned.

        pageNum integer Optional.
        Specifies the page to retrieve search results.

    '''
    def search(self, searchVal, returnGeom='Y', getAddressDetails='Y', pageNum=1):
        endpoint = f"/api/common/elastic/search"
        params = {"searchVal": searchVal,
                  "returnGeom": returnGeom,
                  "getAddrDetails": getAddressDetails,
                  "pageNum": pageNum}

        response = self.__core.make_request(endpoint, params)
        return response


response = OneMap.getToken(email, pw)
access_token = response['access_token']

# Instantiate OneMap object for API query.
onemap = OneMap(access_token)

## Get data on HDB

In [ ]:
df_resale = pd.read_csv("/content/ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv")

In [ ]:
address_val = df_resale['block'] + " " + df_resale['street_name']

In [ ]:
address_set = list(set(address_val.values))

In [ ]:
def getcoordinates(address):
    req = requests.get('https://developers.onemap.sg/commonapi/search?searchVal='+address+'&returnGeom=Y&getAddrDetails=Y&pageNum=1')
    resultsdict = eval(req.text)
    if len(resultsdict['results'])>0:
        return resultsdict['results'][0]['LATITUDE'], resultsdict['results'][0]['LONGITUDE']
    else:
        pass

In [ ]:
response = OneMap.getToken(email, pw)
access_token = response['access_token']

# Instantiate OneMap object for API query.
onemap = OneMap(access_token)
test = onemap.search(address_set[0])

In [ ]:
test

{'found': 1,
 'totalNumPages': 1,
 'pageNum': 1,
 'results': [{'SEARCHVAL': '980C BUANGKOK CRESCENT SINGAPORE 533980',
   'BLK_NO': '980C',
   'ROAD_NAME': 'BUANGKOK CRESCENT',
   'BUILDING': 'NIL',
   'ADDRESS': '980C BUANGKOK CRESCENT SINGAPORE 533980',
   'POSTAL': '533980',
   'X': '33303.1523972359',
   'Y': '40222.472558111',
   'LATITUDE': '1.38003179108977',
   'LONGITUDE': '103.880971331232'}]}

In [ ]:
from tqdm import tqdm
coordinatesdict = {}
count = 0
failed_count = 0
len_address_set = len(address_set)
for address in tqdm(address_set):
    try:
        info = onemap.search(address)
        count = count + 1
        #print('Extracting',count,'out of',len_address_set,'addresses')
        coordinatesdict[address] = info
    except KeyboardInterrupt:
      break
    except:
        count = count + 1
        failed_count = failed_count + 1
        print('Failed to extract',count,'out of',len_address_set,'addresses')
        coordinatesdict[address] = None
print('Total Number of Addresses With No Coordinates',failed_count)

100%|██████████| 2628/2628 [31:07<00:00,  1.41it/s]

Total Number of Addresses With No Coordinates 0


In [ ]:
long_lat_dict = {}
for key, val in coordinatedict.items():
  for v in val['results']:
    if v['POSTAL'] in geojson_des.keys() or v['POSTAL'] == 'NIL':
      long_lat_dict[key] = {'longitude': v['LONGITUDE'], 'latitude': v['LATITUDE']}
      continue

In [ ]:
with open('long_lat_hdb.json', 'w') as fp:
  json.dump(long_lat_dict,fp)

## Calculate distance to mrt station

In [ ]:
df_mrt_loc = gpd.read_file("/content/LTAMRTStationExitGEOJSON.geojson")

In [ ]:
for i in range(df_mrt_loc.shape[0]):
  tree = html.fromstring(df_mrt_loc['Description'][i])
  td = tree.xpath('//td/text()')
  th = tree.xpath('//th/text()')
  temp_data = {h:d for h, d in zip(th, td)}
  pprint(temp_data)
  break

{'EXIT_CODE': 'Exit B',
 'FMEL_UPD_D': '20230127204435',
 'INC_CRC': 'D3065772B4A8884B',
 'STATION_NA': 'KAKI BUKIT MRT STATION'}


In [ ]:
def calc_dist(p1, p2):
  x1 = p1.x
  y1 = p1.y
  x2, y2 = p2.x, p2.y
  return geopy.distance.geodesic((y1, x1), (y2, x2)).km

p2 = Point(*long_lat_dict['216 CHOA CHU KANG CTRL'].values())
calc_dist_partial = partial(calc_dist, p2 = p2)
df_mrt_loc['geometry'].apply(calc_dist_partial).min()

0.3350902502170538

In [ ]:
nearest_mrt_dist = {}
for key, val in tqdm(long_lat_dict.items()):
  p2 = Point(*long_lat_dict[key].values())
  calc_dist_partial = partial(calc_dist, p2 = p2)
  nearest_mrt_dist[key] = df_mrt_loc['geometry'].apply(calc_dist_partial).min()

100%|██████████| 9585/9585 [24:46<00:00,  6.45it/s]


In [ ]:
with open('mrt_dist.json','w') as fp:
  json.dump(nearest_mrt_dist, fp)

## split schools by level and get long lat from api

In [ ]:
df_schools = pd.read_csv('/content/Generalinformationofschools.csv')

In [ ]:
df_schools.head()

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,nature_code,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,na,63627512,na,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,...,CO-ED SCHOOL,FULL DAY,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
1,ADMIRALTY SECONDARY SCHOOL,http://www.admiraltysec.moe.edu.sg,31 WOODLANDS CRESCENT,737916,63651733,63654596,63652774,na,Admiralty_SS@moe.edu.sg,ADMIRALTY MRT,...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
2,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,10 YISHUN STREET 11,768643,67592906,na,67592927,na,aips@moe.edu.sg,Yishun,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
3,AHMAD IBRAHIM SECONDARY SCHOOL,http://www.ahmadibrahimsec.moe.edu.sg,751 YISHUN AVENUE 7,768928,67585384,na,67557778,na,aiss@moe.edu.sg,"CANBERRA MRT, YISHUN MRT",...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
4,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,100 Bright Hill Drive,579646,64547672,na,64532726,na,AITONG_SCH@MOE.EDU.SG,Bishan MRT,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,Yes,No,No,No,Chinese,na,na


In [ ]:
df_schools['mainlevel_code'].unique()

array(['PRIMARY', 'SECONDARY', 'JUNIOR COLLEGE', 'MIXED LEVELS',
       'CENTRALISED INSTITUTE'], dtype=object)

In [ ]:
schools_dict = {}
for vals in df_schools['mainlevel_code'].unique():
  if vals in ['PRIMARY', 'SECONDARY', 'JUNIOR COLLEGE']:
    schools_dict[vals] = []
for i in range(df_schools.shape[0]):
  if df_schools.iloc[i]['mainlevel_code'] == 'MIXED LEVELS':
    schools_dict['SECONDARY'].append(df_schools.iloc[i])
    if 'Sec' in df_schools.iloc[i]['telephone_no']:
      schools_dict['PRIMARY'].append(df_schools.iloc[i])
    else:
      schools_dict['JUNIOR COLLEGE'].append(df_schools.iloc[i])
  elif df_schools.iloc[i]['mainlevel_code'] == 'CENTRALISED INSTITUTE':
    schools_dict['JUNIOR COLLEGE'].append(df_schools.iloc[i])
  else:
    schools_dict[df_schools.iloc[i]['mainlevel_code']].append(df_schools.iloc[i])

In [ ]:
school_geo_info = {}
count = 0
failed_count = 0
for key, val in schools_dict.items():
  for v in tqdm(val):
      try:
          info = onemap.search(v['postal_code'])
          count = count + 1
          #print('Extracting',count,'out of',len_address_set,'addresses')
          school_geo_info[v['school_name']] = info
      except KeyboardInterrupt:
        break
      except:
          count = count + 1
          failed_count = failed_count + 1
          print('Failed to extract',count)
          school_geo_info[v['school_name']] = None
print('Total Number of Addresses With No Coordinates',failed_count)

100%|██████████| 22/22 [00:12<00:00,  1.78it/s]

Total Number of Addresses With No Coordinates 0


In [ ]:
with open('School_onemap.json', 'w') as fp:
  json.dump(school_geo_info, fp)

In [ ]:
# schools dict key is ['PRIMARY', 'SECONDARY', 'JUNIOR COLLEGE'], values is list of pd.Series
for key, vals in schools_dict.items():
  for i in range(len(vals)):
    schools_dict[key][i] = schools_dict[key][i].to_dict()

In [ ]:
with open('School_by_level.json', 'w') as fp:
  json.dump(schools_dict, fp)

## Extract Latitude and longitude of schools

In [ ]:
with open('School_onemap.json', 'r') as fp:
  school_onemap = json.load(fp)

In [ ]:
sch_long_lat = {}
for key, val in tqdm(school_onemap.items()):
  try:
    sch_long_lat[key] = {}
    info = school_onemap[key]['results'][0]
    sch_long_lat[key]['LONGITUDE'] = info['LONGITUDE']
    sch_long_lat[key]['LATITUDE'] = info['LATITUDE']
  except IndexError:
    # Only school where data is missing
    info = onemap.search('ZHENGHUA SECONDARY SCHOOL')['results'][0]
    sch_long_lat[key]['LONGITUDE'] = info['LONGITUDE']
    sch_long_lat[key]['LATITUDE'] = info['LATITUDE']

100%|██████████| 337/337 [00:00<00:00, 485.81it/s]


In [ ]:
with open('/content/School_by_level.json', 'r') as fp:
  sch_level = json.load(fp)

In [ ]:
sch_name = {}
for key, val in sch_level.items():
  # sch_name has key ['PRIMARY', 'SECONDARY', 'JUNIOR COLLEGE'], value is list of school names
  sch_name[key] = [v['school_name'] for v in val]

In [ ]:
sch_lat_long_by_level = {}
for key, val in sch_name.items():
  sch_lat_long_by_level[key] = {}
  for sch in val:
    sch_lat_long_by_level[key][sch] = sch_long_lat[sch]

In [ ]:
with open('sch_long_lat_by_level.json', 'w') as fp:
  json.dump(sch_lat_long_by_level, fp)

## Get distance per school level to HDB

In [ ]:
def calc_dist(p1, p2):
  x1 = p1.x
  y1 = p1.y
  x2, y2 = p2.x, p2.y
  return geopy.distance.geodesic((y1, x1), (y2, x2)).km

In [ ]:
with open('/content/long_lat_hdb.json', 'r') as fp:
  long_lat_hdb_dict = json.load(fp)

with open('/content/sch_long_lat_by_level', 'r') as fp:
  long_lat_sch_dict = json.load(fp)

In [ ]:
nearest_sch_dist = {}
closeness_of_top_sch = {}
for key, val in tqdm(long_lat_hdb_dict.items()):
  try:
    nearest_sch_dist[key] = {}
    p2 = Point(*long_lat_hdb_dict[key].values())
    calc_dist_partial = partial(calc_dist, p2 = p2)
    for k, v in long_lat_sch_dict.items():
      nearest_sch_dist[key][k] = {sch_name: calc_dist_partial(Point(*v1.values())) for sch_name, v1 in v.items()}
      nearest_sch_dist[key][k] = sorted(nearest_sch_dist[key][k].items(), key = lambda item: item[1])[0]
  except KeyboardInterrupt:
    break
  except Exception as e:
    print('ERROR', e)

100%|██████████| 9585/9585 [09:07<00:00, 17.50it/s]


In [ ]:
with open('nearest_sch_dist.json', 'w') as fp:
  json.dump(nearest_sch_dist, fp)

## Adjust Price

In [ ]:
df_price_index = pd.read_csv('/content/HDBResalePriceIndex1Q2009100Quarterly.csv')

In [ ]:
df_resale = pd.read_csv('/content/ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv')

In [ ]:
def foo(month):
  base = index_dict[2017][0]
  year, month = month.split('-')
  year = int(year)
  month = int(month) - 1
  return base / index_dict[year][month // 3]

(df_resale['month'].apply(foo) * df_resale['resale_price'])

<ipython-input-49-7fe77b0d539a>:6: RuntimeWarning: divide by zero encountered in scalar divide
  return base / index_dict[year][month // 3]


,0
0,232000.0
1,250000.0
2,262000.0
3,265000.0
4,265000.0
...,...
191231,inf
191232,inf
191233,inf
191234,inf


In [ ]:
df_inflation_adj_price = df_resale['month'].apply(foo) * df_resale['resale_price']
df_resale['adjusted_price'] = df_inflation_adj_price

<ipython-input-49-7fe77b0d539a>:6: RuntimeWarning: divide by zero encountered in scalar divide
  return base / index_dict[year][month // 3]


In [ ]:
df_resale.to_csv('Resaleflatprices_adjusted.csv', index = False)

## Get long lat of Parks

In [ ]:
df_park_loc = gpd.read_file("/content/Parks.geojson")
df_nat_res_loc = gpd.read_file("/content/NParksParksandNatureReserves.geojson")
with open("/content/long_lat_hdb.json", 'r') as fp:
  lat_long_dict = json.load(fp)

In [ ]:
df_park_loc.head()

,Name,Description,geometry
0,kml_1,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.78118 1.31567 0)
1,kml_2,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.78275 1.31251 0)
2,kml_3,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.7826 1.32677 0)
3,kml_4,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.75723 1.37588 0)
4,kml_5,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.77165 1.33584 0)


In [ ]:
with open('long_lat_hdb.json', 'r') as fp:
  long_lat_dict = json.load(fp)

def calc_dist(p1, p2):
  x1 = p1.x
  y1 = p1.y
  x2, y2 = p2.x, p2.y
  return geopy.distance.geodesic((y1, x1), (y2, x2)).km

p2 = Point(*long_lat_dict['216 CHOA CHU KANG CTRL'].values())
calc_dist_partial = partial(calc_dist, p2 = p2)
shapely.centroid(df_nat_res_loc['geometry']).apply(calc_dist_partial).min()

0.4424440216646317

In [ ]:
nearest_park_dist = {}
for key, val in tqdm(long_lat_dict.items()):
  p2 = Point(*long_lat_dict[key].values())
  calc_dist_partial = partial(calc_dist, p2 = p2)
  nearest_park_dist[key] = df_park_loc['geometry'].apply(calc_dist_partial).min()

100%|██████████| 9585/9585 [20:13<00:00,  7.90it/s]


In [ ]:
for key, val in tqdm(long_lat_dict.items()):
  p2 = Point(*long_lat_dict[key].values())
  calc_dist_partial = partial(calc_dist, p2 = p2)
  nearest_park_dist[key] = min(nearest_park_dist[key], shapely.centroid(df_nat_res_loc['geometry']).apply(calc_dist_partial).min())

100%|██████████| 9585/9585 [19:55<00:00,  8.02it/s]


In [ ]:
nearest_park_dist

{'980C BUANGKOK CRES': 0.6633018675861376,
 '615 WOODLANDS AVE 4': 1.9592514347514165,
 '417 EUNOS RD 5': 0.650961726514336,
 '455A ANG MO KIO ST 44': 0.5042180870524673,
 '143 SERANGOON NTH AVE 1': 0.5369804471421065,
 '341 WOODLANDS AVE 1': 0.6041314955374384,
 '119 HO CHING RD': 0.19499649926749654,
 '624 JURONG WEST ST 61': 0.6595195028402631,
 '289 CHOA CHU KANG AVE 3': 1.1260909202481357,
 '266A COMPASSVALE BOW': 0.6184991990257768,
 '511 HOUGANG AVE 10': 0.49492902937419,
 '17B CIRCUIT RD': 0.33038841803547037,
 '227 YISHUN ST 21': 0.3381241017897224,
 '364A UPP SERANGOON RD': 0.45613375630642056,
 '250 HOUGANG AVE 3': 0.7675739683543501,
 '588A ANG MO KIO ST 52': 0.2991316655984972,
 '209C COMPASSVALE LANE': 0.7578705013050601,
 '237 CHOA CHU KANG CTRL': 0.849006645329323,
 '439 CHOA CHU KANG AVE 4': 0.9232949892564605,
 '343 CLEMENTI AVE 5': 0.6811617769229044,
 '40 TELOK BLANGAH RISE': 0.3862775703871818,
 '547 BEDOK NTH ST 3': 0.5302744534703144,
 '62 CIRCUIT RD': 0.49559277

In [ ]:
with open('park_dist.json','w') as fp:
  json.dump(nearest_park_dist, fp)

## Get long lat of Sports Facilities

In [ ]:
df_sport_field_loc = gpd.read_file("/content/SportsFieldsSG.geojson")
df_sport_facil_loc = gpd.read_file("/content/SportSGSportFacilitiesGEOJSON.geojson")

with open("/content/long_lat_hdb.json", 'r') as fp:
  long_lat_dict = json.load(fp)

In [ ]:
def calc_dist(p1, p2):
  x1 = p1.x
  y1 = p1.y
  x2, y2 = p2.x, p2.y
  return geopy.distance.geodesic((y1, x1), (y2, x2)).km


nearest_sport_dist = {}
for key, val in tqdm(long_lat_dict.items()):
  p2 = Point(*long_lat_dict[key].values())
  calc_dist_partial = partial(calc_dist, p2 = p2)
  nearest_sport_dist[key] = df_sport_field_loc['geometry'].apply(calc_dist_partial).min()
  nearest_sport_dist[key] = min(nearest_sport_dist[key], shapely.centroid(df_sport_facil_loc['geometry']).apply(calc_dist_partial).min())

100%|██████████| 9585/9585 [10:10<00:00, 15.69it/s]


In [ ]:
min(nearest_sport_dist.values())

0.00039544536300965946

In [ ]:
with open('sport_dist.json','w') as fp:
  json.dump(nearest_sport_dist, fp)

# Compile Data

In [ ]:
df_resale = pd.read_csv("/content/Resaleflatprices_adjusted.csv")

In [ ]:
with open('/content/park_dist.json', 'r') as fp:
  park_dist = json.load(fp)
with open('/content/sport_dist.json', 'r') as fp:
  sport_dist = json.load(fp)
with open('/content/nearest_sch_dist.json', 'r') as fp:
  school_dist = json.load(fp)
with open('/content/mrt_dist.json', 'r') as fp:
  mrt_dist = json.load(fp)

In [ ]:
_dict = {}
_dict['distance_to_mrt'] = []
_dict['distance_to_park'] = []
_dict['distance_to_sports_facilities'] = []
_dict['distance_to_primary_school'] = []
_dict['distance_to_secondary_school'] = []
_dict['distance_to_junior_college'] = []
for address in tqdm(address_val):
  _dict['distance_to_mrt'].append(mrt_dist[address])
  _dict['distance_to_park'].append(park_dist[address])
  _dict['distance_to_sports_facilities'].append(sport_dist[address])
  _dict['distance_to_primary_school'].append(school_dist[address]['PRIMARY'])
  _dict['distance_to_secondary_school'].append(school_dist[address]['SECONDARY'])
  _dict['distance_to_junior_college'].append(school_dist[address]['JUNIOR COLLEGE'])

100%|██████████| 191236/191236 [00:00<00:00, 387348.32it/s]


In [ ]:
df_new = pd.concat((df_resale, pd.DataFrame(_dict)), axis = 1)

In [ ]:
df_new.to_csv('Combined_df.csv')

In [ ]:
df_new.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,adjusted_price,distance_to_mrt,distance_to_park,distance_to_sports_facilities,distance_to_primary_school,distance_to_secondary_school,distance_to_junior_college
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,232000.0,0.934249,0.668796,0.464563,0.227903,0.465767,1.401613
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,250000.0,0.136278,0.263330,0.403312,0.257084,0.414490,1.041037
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,262000.0,0.390204,0.131935,0.790995,0.506849,0.789166,1.172924
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,265000.0,0.885083,0.371512,0.545979,0.691690,0.518559,1.827234
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,265000.0,0.376976,0.151297,0.830485,0.546444,0.810812,1.205414
